In [14]:
import pandas as pd
pd.set_option('display.max_rows', 500)
import plotly.express as px
from pathlib import Path
from dataclasses import dataclass

# Point 4: Accelerating the Shift to Zero Emission Vehicles, 
End the sale of new pure 
petrol and diesel cars and 
vans by 2030 and consult on 
phase out for diesel HGVs

## What % of cars on the road today are Zero Emission?

In [2]:
make_model_df = pd.read_csv(Path("raw_data", "by_make_model.csv"))
make_model_df = make_model_df[make_model_df['LicenceStatus'] == 'Licensed']

quarters_cols = ['2022Q3',
       '2022Q2', '2022Q1', '2021Q4', '2021Q3', '2021Q2', '2021Q1', '2020Q4',
       '2020Q3', '2020Q2', '2020Q1', '2019Q4', '2019Q3', '2019Q2', '2019Q1',
       '2018Q4', '2018Q3', '2018Q2', '2018Q1', '2017Q4', '2017Q3', '2017Q2',
       '2017Q1', '2016Q4', '2016Q3', '2016Q2', '2016Q1', '2015Q4', '2015Q3',
       '2015Q2', '2015Q1', '2014Q4', '2014Q3', '2014Q2', '2014Q1', '2013Q4',
       '2013Q3', '2013Q2', '2013Q1', '2012Q4', '2012Q3', '2012Q2', '2012Q1',
       '2011Q4', '2011Q3', '2011Q2', '2011Q1', '2010Q4', '2010Q3', '2010Q2',
       '2010Q1', '2009Q4', '2009Q3', '2009Q2', '2009Q1', '2008Q4', '2008Q3',
       '2007Q4', '2006Q4', '2005Q4', '2004Q4', '2003Q4', '2002Q4', '2001Q4',
       '2000Q4', '1999Q4', '1998Q4', '1997Q4', '1996Q4', '1995Q4', '1994Q4']

# make_model_df['total'] = make_model_df.sum(numeric_only = True, axis = 1)

make_model_df = make_model_df[quarters_cols].T
make_model_df['total'] = make_model_df.sum(numeric_only = True, axis = 1)
make_model_df = make_model_df.iloc[::-1] #Reversing index for chronological order
make_model_df
fig = px.line(make_model_df, x=make_model_df.index, y="total", title='Total vehicles licensed in the UK', template='plotly_dark', 
              labels={
                     "index": "Date",
                     "total": "Num. Vehicles",
                 },)
fig.show()

In [3]:
make_model_df = make_model_df.reset_index()
make_model_df.head()
make_model_df = make_model_df[['index', 'total']]
make_model_df['date_stripped' ] = make_model_df['index'].str.slice(0,4) + " " + make_model_df['index'].str.slice(4, 6)
make_model_df= make_model_df.rename(columns = {'total':'total licensed vehicles'})
make_model_df = make_model_df.drop('index', axis = 1)

In [4]:
piv_df_raw = pd.read_excel(Path("raw_data", "veh0141_PIV.ods"), engine="odf", sheet_name='VEH0141a_Fuel', skiprows = 4)
piv_df = piv_df_raw.copy()

In [5]:
piv_df = piv_df[piv_df['Geography [note 2]'] == 'United Kingdom']
piv_df = piv_df[piv_df['Units'] == 'Number']
piv_df = piv_df.drop(['Geography [note 2]', 'Units', 'Battery Electric','Plug-in hybrid electric (petrol)',	'Plug-in hybrid electric (diesel)',	'Range extended electric'], axis = 1)

In [6]:
def strip_date(string:str):
    years = []
    for i, _ in enumerate(range(150)):
        year = 1900 + i
        years.append(str(year)) 
    quarters = ["Q1", "Q2", "Q3", "Q4"]
    to_keep = years + quarters
    return ' '.join([x for x in string.split() if x in to_keep])

In [7]:
piv_df['date_stripped'] = piv_df['Date'].apply(lambda x: strip_date(x))
piv_df['Total'] = piv_df['Total'].astype(float)
piv_df_grouped = piv_df.groupby('date_stripped').sum(numeric_only = True)

In [8]:
fig = px.line(piv_df_grouped, x=piv_df_grouped.index, y="Total", title='Plug in vehicles licensed in the UK', template='plotly_dark', 
                            labels={
                     "date_stripped": "Date",
                     "Total": "Num. Vehicles",
                 }) # This is all vehicles - HGV, Buses, motorbikes, cars
fig.show()

In [9]:
piv_df_grouped = piv_df_grouped.rename(columns = {'Total':'total plug in vehicles licensed'}).reset_index()

In [10]:
merged_df = make_model_df.merge(piv_df_grouped, how = 'left')

In [46]:
fig = px.line(merged_df, x='date_stripped',
              y=["total plug in vehicles licensed","total licensed vehicles"], 
              title='Plug in vehicles licensed in the UK', 
              template='plotly_dark', 
               labels={
                     "date_stripped": "Date",
                     "value": "Num. Vehicles",
                 }) # This is all vehicles - HGV, Buses, motorbikes, cars
fig.update_layout(legend_title_text=' ')
fig.update_layout(legend=dict(
    yanchor="top",
    y=1.4,
    xanchor="left",
    x=0.4
))
fig.show()

Back to the original question - what % of cars today are zero emission? Note the only data available is for plug in cars, not necessarily zero emission (includes hybrids)

In [12]:
merged_df['% Plug in vehicles'] = 100*(merged_df['total plug in vehicles licensed'] / merged_df['total licensed vehicles'])

In [13]:
merged_df.tail()

,total licensed vehicles,date_stripped,total plug in vehicles licensed,% Plug in vehicles
66,39236290,2021 Q3,1302632.0,3.319967
67,39034266,2021 Q4,1495790.0,3.831992
68,39123642,2022 Q1,1686476.0,4.310631
69,39430211,2022 Q2,1843810.0,4.676135
70,39519623,2022 Q3,2006144.0,5.076324


## What % of cars over the last 5 years sold have been zero emission?

## Whats the average cost of a zero emission car compared to petrol/diesel?

The cost of a new zero emisison car is a big hurdle for a lot of buyers, as these are typically much more expnsive than a low cost petrol or diesel car. Although savings can be made up through lower cost of fuel, the government may need to implement a help to buy system if they want people buying more electric. 

In [27]:
# https://www.edfenergy.com/energywise/cheapest-electric-cars-to-buy
#https://www.autoexpress.co.uk/best-cars-vans/351901/top-10-cheapest-cars-sale-2023
@dataclass
class Car:
    name: str
    price: int

@dataclass
class ElectricCar(Car):
    range: int
    
vw_id3 = ElectricCar("VW ID.3 Life Pro Performance", 29990, 265)
peugeot_e_208 = ElectricCar("Peugeot e-208", 30195, 217)
vauxhall_corsa_e = ElectricCar("Vauxhall Corsa-e", 29995, 209)
nissan_leaf_acenta = ElectricCar("Nissan Leaf Acenta", 28995, 168)
mg_zs_ev = ElectricCar("MG ZS EV", 18995, 165)
mg4_ev = ElectricCar("MG4 EV", 25995, 218)
ds3_crossback_etense = ElectricCar("DS3 Crossback E-Tense", 33935, 191)
vw_e_up = ElectricCar("VW e-Up", 23555, 140)
mini_electric = ElectricCar("MINI Electric", 28000, 140)
fiat_500e = ElectricCar("Fiat 500e", 23495, 115)

dacia_sandero = Car('Dacia Sandero',12595)
mg3 = Car('MG3',13295)
kia_picanto = Car('Kia Picanto',13400)
hyundai_i10 = Car('Hyundai i10',13430)
dacia_sandero_stepway = Car('Dacia Sandero Stepway',13795)
citroen_c3 = Car('Citroen C3',13995)
volkswagen_up = Car('Volkswagen up!',14070)
fiat_panda = Car('Fiat Panda',14485)
fiat_500_hybrid = Car('Fiat 500 hybrid',14990)
dacia_duster = Car('Dacia Duster',15295)
fiat_panda = Car('Fiat Panda',14485)
fiat_500_hybrid = Car('Fiat 500 hybrid',14990)
dacia_duster = Car('Dacia Duster',15295)



Lets compare the price of the cheapest electric cars on sale in the UK today with the cheapest petrol/diesel cars available

In [32]:
electric_car_list = [vw_id3,
peugeot_e_208,
vauxhall_corsa_e,
nissan_leaf_acenta,
mg_zs_ev,
mg4_ev,
ds3_crossback_etense,
vw_e_up,
mini_electric,
fiat_500e]

car_list = [dacia_sandero,
mg3,
kia_picanto,
hyundai_i10,
dacia_sandero_stepway,
citroen_c3,
volkswagen_up,
fiat_panda,
fiat_500_hybrid,
dacia_duster,
fiat_panda,
fiat_500_hybrid,
dacia_duster,
]

In [54]:
electric_df = pd.DataFrame(electric_car_list).sort_values(by = 'price')
electric_df['type'] = 'Electric'
petrol_diesel_df = pd.DataFrame(car_list).sort_values(by = 'price')
petrol_diesel_df['type'] = 'Petrol/Diesel'

car_df = pd.concat([electric_df, petrol_diesel_df]).drop_duplicates()


In [57]:
fig = px.bar(car_df, x='name', y='price', 
             title='Electric and Petrol/Diesel Car Prices', 
             color = 'type',
              template='plotly_dark', 
              labels={
                     "name": "Name",
                     "price": "Price (£)",
                 })
fig.update_layout(legend_title_text=' ')
fig.update_layout(legend=dict(
    yanchor="top",
    y=1.45,
    xanchor="left",
    x=0.25
))
fig.show()

The cheapest electric car is £4000 more expensive than the most expensive petrol/diesel car.

Let's see how price can impact range on these budget electric cars. 

In [62]:
fig = px.scatter(car_df[car_df['type'] == 'Electric'], x='range', y='price', 
             title='Budget Electric car Price vs Range', 
             color = 'type',
              template='plotly_dark', 
              labels={
                     "range": "Range (Miles)",
                     "price": "Price (£)",
                 })

fig.show()